# Technician Routing and Scheduling Problem

- 온디멘드 세차 서비스를 제공하는 '(주)인스타워시'와의 프로젝트 결과물입니다.
- 테크니션 과학적 배치 및 경로 최적화를 통한 수익 개선이 목적으로, 
네이버 API 연동 및 Gurobi MIP(Mixed Interger Programming) 최적화모델 커스터마이징하여 적용하였습니다. 
- 시뮬레이션 최종 결과는 20-30%의 이동경로 효율향상과 10%의 매출증감 효과가 나타났습니다. 
- 현재 추가 고도화 및 on site test가 3월에 예정돼 있습니다. 

#### * 본 프로젝트는 Gurobi의 technician Routing and Scheduling Problem 예제를 인스타워시 상황에 맞게 modify하여 적용하였습니다.

### Sets and Indices
$j \in J = \{1,2,...,n\}$: Index and set of jobs.

$k \in K$: Index and set of technicians.

$K(j) \subset K$: Subset of technicians qualified to perform job $j \in J$ 

$d \in D = \{n+1, n+2, ...,n+m \}$: Index and set of depots (service centers), where $m$ is the number of depots.

$l, i, j \in L = J \cup D = \{1,2,..., n+m \}$: index and set of locations to visit.

### Parameters
$\beta_{k,d} \in \{0,1\}$: This parameter is equal 1 if technician $k \in K$ must depart from and return to depot 
$d \in D$; and 0 otherwise.

$p_{j} \in \mathbb{R^{+}}$; Processing time (duration) of job $j \in J$.

$\tau_{i,j} \in \mathbb{R^{+}}$: Travel time, in minutes, from location $i \in L$ to location $j \in L$.

$W_{k} \in \mathbb{N}$: Workload limit of technician $k \in K$ during the planning horizon. That is, the number of hours, in minutes, that the technician is available for the next work day.

$\pi_{j} \in \{1,2,3,4\}$: Priority weight of job $j \in J$, a larger number means higher priority.

$a_{j} \in \mathbb{R^{+}} $: Earliest time to start the service at location of job $j \in J$.

$b_{j} \in \mathbb{R^{+}} $: Latest time to start the service at location of job $j \in J$.

$dd_{j} \in \mathbb{R^{+}} $: Due date of job $j \in J$, i.e. the latest time to complete the service for job $j \in J$.

$M \in \mathbb{N}$: This is a very large number. This number is determined as follows: the planning horizon is of 10 working hours (i.e. 600 min). We want the value of $M$ to be an order of magnitude larger than the planning horizon. Therefore, we set M = 6100.

### Decision Variables

$x_{j,k} \in \{0,1\}$: This variable is equal 1 if job $j \in J$ is assigned to technician $k \in K$, and 0 otherwise.

$u_{k} \in \{0,1\}$: This variable is equal 1 if technician $k \in K$  is used to perform a job, and 0 otherwise.

$y_{i,j,k} \in \{0,1\}$: This variable is equal 1 if technician $k \in K$  travels from location $i \in L$ to location 
$j \in L$; and 0 otherwise.

$t_{j} \geq 0$: This variable determines the time to arrive or start the service at location $j \in J$.

$z_{j} \geq 0$: This variable determines the lateness of completing job $j \in J$.

$xa_{j}, xb_{j} \geq 0$: Correction to earliest and latest time to start the service for job $j \in J$.

$g_{j}$: This variable is equal 1 if job $j \in J$ cannot be filled, and 0 otherwise. 

### Objective Function

- **Minimize lateness:** The Objective function is to minimize the total weighted lateness of all the jobs. 

\begin{equation}
\sum_{j \in J} \pi_{j} \cdot z_{j} + \sum_{j \in J} 0.01 \cdot \pi_{j} \cdot M(xa_{j} + xb_{j}) + 
\sum_{j \in J} \pi_{j} \cdot M \cdot g_{j}
\tag{0}
\end{equation}

**Note**: We want to treat the constraints of filling the jobs demand and the constraints about starting the jobs within a time window as soft constraints. Soft constraints can be violated, but the violation will incur in a huge penalty. To consider that each job has different priority, we use the priority penalty for the calculation of the penalty associated with a violation of soft constraints. We assume that not filling jobs demand greatly deteriorate customer satisfaction, consequently should incur the largest penalties associated with the soft constraints. Recall that the value of the paramter $M$ is a large number, then the penalty associated with not filling a job is determined as follows: $\pi_{j} \cdot M$. The time windows constraints violation also deteriorates customer satisfaction but to a lesser degree than the penalty associated with violating the time windows constraints. The time window constraints are determined as follows: $0.01 \cdot \pi_{j} \cdot M$.

### Constraints

- **Assign qualified technicians:** For each job, we  assign one technician who is qualified for the job, or a gap is declared. 

\begin{equation}
\sum_{k \in K(j)} x_{j,k} + g_{j} = 1 \quad \forall j \in J
\tag{1}
\end{equation}

**Note**: The penalty of the gap variable $g_{j}$ is ($0.1 \cdot \pi_{j} \cdot M$) which is a large number to discourage 
not being able to satisfy demand.

- **Only one technician:** For each job, we only allow one technician to be assigned.

\begin{equation}
\sum_{k \in K} x_{j,k} \leq 1 \quad \forall j \in J
\tag{2}
\end{equation}

- **Technician capacity:** For each technician, we ensure that the available capacity of the technician is not exceeded. 

\begin{equation}
\sum_{j \in J}  p_{j} \cdot x_{j,k} + \sum_{i \in L} \sum_{j \in L} \tau_{i,j} \cdot y_{i,j,k} \leq W_{k} \cdot u_{k} \quad \forall k \in K
\tag{3}
\end{equation}

- **Technician tour:** For each technician and job, we ensure that if the technician is assigned to the job, then the technician must travel to another location (to form a tour). 

\begin{equation}
\sum_{j \in L}  y_{i,j,k} = x_{i,k} \quad \forall i \in J, k \in K
\tag{4}
\end{equation}

- For each technician and job, we ensure that if a technician is assigned to the job, then the technician must travel from another location to the location of the job (to form a tour). 

\begin{equation}
\sum_{i \in L}  y_{i,j,k} = x_{j,k} \quad \forall j \in J, k \in K
\tag{5}
\end{equation}

- **Same depot:** For each technician and depot, we ensure that a technician, if assigned to any job, must depart from and return to the service center (depot) where the technician is based. 

\begin{equation}
\sum_{j \in J}  y_{d,j,k} = \beta_{k,d} \cdot u_{k} \quad \forall  k \in K, d \in D
\tag{6}
\end{equation}

\begin{equation}
\sum_{i \in J}  y_{i,d,k} = \beta_{k,d} \cdot u_{k} \quad \forall  k \in K, d \in D
\tag{7}
\end{equation}

- **Temporal relationship:** For each location and job, we ensure the temporal relationship between two consecutive jobs served by the same technician. That is, if a technician $k$ travels from job $i$ to job $j$, then the start of the service time at job $j$ must be no less than the completion time of job $i$ plus the travel time from job $i$ to job $j$. 

\begin{equation}
t_{j} \geq t_{i} + p_{i} + \tau_{i,j} - M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) \quad \forall i \in L, j \in J
\tag{8}
\end{equation}

**Note**: Observe that if the technician $k$ travels from the location of job $i$ to the location of job $j$, then
$\sum_{k \in K}  y_{i,j,k} = 1$. Therefore, $M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) = 0$, and the constraint 
$t_{j} \geq t_{i} + p_{i} + \tau_{i,j}$ would be properly enforced. Now consider the case where the technician $k$ does not travel  from the location of job $i$ to the location of job $j$. Hence, $\sum_{k \in K}  y_{i,j,k} = 0$ and 
$M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) = M$. In this case, this constraint becomes 
$t_{j} \geq t_{i} + p_{i} + \tau_{i,j} - M$. But $M$ is a very large number, then $t_{i} + p_{i} + \tau_{i,j} - M < 0$ and 
since $t_{j} \geq 0$, this constraint is redundant.

- **Time window:** For each job $j \in J$ ensure that the time window for the job is satisfied.

\begin{equation}
t_{j} \geq a_{j} - xa_{j} \quad \forall j \in J
\tag{9}
\end{equation}

\begin{equation}
t_{j} \leq b_{j} + xb_{j} \quad \forall j \in J
\tag{10}
\end{equation}

**Note**: To discourage that the time window of a job is violated, we associate the  penalty of 
($0.01 \cdot \pi_{j} \cdot M$) to the correction variables $xa_{j}, xb_{j}$.

- **Lateness constraint:** For each job $j \in J$ calculate the lateness of the job.

\begin{equation}
z_{j} \geq t_{j} + p_{j} - dd_{j} \quad \forall j \in J
\tag{11}
\end{equation}

- Note that since the lateness decision variable $z_{j}$ is non-negative, there is no benefit to complete a job before its due date; on the other hand, since the objective function minimizes the total weighted lateness, Constraint (11) should always be binding.

In [1]:
# 패키지 loading 
#import random
import pandas as pd
import numpy as np
from urllib.request import urlopen, Request
from urllib import parse 
from urllib.error import HTTPError 
from bs4 import BeautifulSoup 
import json 
import requests

from geopy.geocoders import Nominatim
import pymysql
import sys
from collections import defaultdict
import xlrd
from gurobipy import GRB
import gurobipy as gp

import time
from tqdm import tqdm_notebook

### SQL 데이터베이스 접속

In [2]:
# # 패키지 불러오기
# import pymysql

# pd.options.display.max_rows = 1000
# pd.set_option('display.max_columns', None)

# # SQL접속
# conn = pymysql.connect(host='34.64.180.248',
#                        user=***,
#                        password='***,
#                        db=***)

# curs = conn.cursor(pymysql.cursors.DictCursor)

# # 쿼리
# sql = "SELECT TB_USER_ID, TB_USER_ID_EMPLOYEE, ORDER_NO, ORDER_START_DATE, ORDER_END_DATE,ADJUST_PRICE, \
# CAST(AES_DECRYPT (USER_LAT, 'INSTA' ) AS DECIMAL(10,6)) as USER_LAT, \
# CAST(AES_DECRYPT (USER_LNG, 'INSTA' ) AS DECIMAL(10,6)) as USER_LNG,\
# total_duration, price, Status FROM tb_order;"

# # 실행
# curs.execute(sql)

# result = curs.fetchall()
# result = pd.DataFrame(result)

# result.tail()

# # 데이터 백업
# df_order = result.copy()

# df_order = df_order[(df_order['Status'] == '03') | (df_order['Status'] == '04')]

# # datetime타입변환
# df_order["ORDER_START_DATE"] = pd.to_datetime(df_order["ORDER_START_DATE"])
# df_order["ORDER_END_DATE"] = pd.to_datetime(df_order["ORDER_END_DATE"])

# # 예약 시간범위설정 
# time_window_a = "2020-09-26 10:00:00"
# time_window_b = "2020-09-26 18:30:00"

# # 정시선택 
# df_order = df_order[(df_order.ORDER_START_DATE >= time_window_a)
#                     & (df_order.ORDER_START_DATE <= time_window_b)]

# # 시간순 정렬
# df_order = df_order.sort_values(by='ORDER_START_DATE', ascending=True)

# # 서울 근교지역 위도경도 필터링 
# df_order = df_order.loc[(df_order["USER_LAT"] < 37.787346)
#              & (df_order["USER_LAT"] > 37.023035) &
#              (df_order["USER_LNG"] < 127.500868) &
#              (df_order["USER_LNG"] > 126.309627)]

# # 고객의 수 
# cust_number = len(df_order["ORDER_END_DATE"])
# # 테크니션 수
# tech_number = df_order.TB_USER_ID_EMPLOYEE.nunique()
# print(f"총 고객의 수 : {cust_number} 명")
# print(f"총 테크니션 수 : {tech_number} 명")

#### 인스타워시측과의 기밀유지협약으로 현재 포트폴리오에서는 원본대신 샘플 데이터를 사용하였습니다.  

In [3]:
# 샘플데이터 불러오기 
df_order = pd.read_csv("desktop/sample_data_insta_r2.csv")
df_order

,TB_USER_ID,TB_USER_ID_EMPLOYEE,ORDER_START_DATE,USER_LAT,USER_LNG,total_duration,price
0,1,144107,2020.9.26 10:00,37.574830,127.119744,100,45000
1,2,11550,2020.9.26 10:00,37.526000,127.130000,130,48500
2,3,78468,2020.9.26 10:00,37.613276,127.073093,55,38000
3,4,157202,2020.9.26 10:00,37.501808,126.917992,185,105600
4,5,145757,2020.9.26 10:00,37.624488,126.924749,85,42000
...,...,...,...,...,...,...,...
77,78,122639,2020.9.26 17:40,37.517410,126.927296,55,40000
78,79,4874,2020.9.26 18:00,37.266899,127.056862,50,32000
79,80,152510,2020.9.26 18:00,37.486597,127.014798,50,32000
80,81,145757,2020.9.26 18:00,37.560107,126.964631,50,32000


### 주소생성

In [4]:
# 네이버 API로 집주소 위치 확인하기
# 네이버 API로 예측하기
# 네이버 API 사용
# 위도, 경도축출

distance = []
duration = []
index = []
situ = []
address1 = []
address2 = []

# 주소변수 생성
df_order["address"] = np.nan
df_order["address2"] = np.nan

# Naver api 변경필요 
client_id = "5nq2hgq4ct"
client_pw = "eAcaE6E3aJaqTTNK1dtZaQy6XkmzwtcwQGDxul0H"

cnt = 0

for k, i in tqdm_notebook(zip(df_order["USER_LAT"], df_order["USER_LNG"])):
    try:

        naver_url = "https://naveropenapi.apigw.ntruss.com/map-reversegeocode/v2/gc"
        gc_url = f"?request=coordsToaddr&coords={i},{k}&sourcecrs=epsg:4326&orders=admcode,legalcode,addr,roadaddr&output=json"
        sum_url = naver_url + gc_url
        naver_headers = {
            "X-NCP-APIGW-API-KEY-ID": client_id,
            "X-NCP-APIGW-API-KEY": client_pw
        }

        naver_api_test = requests.get(sum_url, headers=naver_headers)
        naver_url_text = json.loads(naver_api_test.text)

        address1.append(
            naver_url_text['results'][0]["region"]["area2"]["name"])
        address2.append(
            naver_url_text['results'][0]["region"]["area3"]["name"])

        df_order["address"][cnt] = naver_url_text['results'][0]["region"][
            "area2"]["name"]
        df_order["address2"][cnt] = naver_url_text['results'][0]["region"][
            "area3"]["name"]

        #print(cnt, df_order["address"][cnt], df_order["address2"][cnt])
        cnt += 1
   
    except:
        continue
        
# 주소삽입 
df_order["address"] = address1
df_order["address2"] = address2

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
//anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
# 지역구분 dictionary 
reg_div = pd.read_csv("desktop/지역구분.csv", header= None)
reg = {i:x for i, x in zip(reg_div[0], reg_div[1])}
reg = pd.concat([pd.DataFrame(reg_div[0]), reg_div[1].str.split(',', expand = True)], axis = 1)
reg.columns = ["지역", 0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
reg = pd.DataFrame(reg.set_index("지역").stack()).reset_index()[["지역", 0]]
reg = {k:i for i, k in zip(reg["지역"], reg[0])}

# 고객지역 매칭 
df_order["지역구분"] = df_order["address"].map(reg)
df_order["지역구분"].value_counts()

print(f"지역배치 결과: {reg}")

지역배치 결과: {'강남구': '1지역', '서초구': '1지역', '송파구': '1지역', '도봉구': '2지역', '강북구': '2지역', '노원구': '2지역', '중랑구': '2지역', '강동구': '2지역', '광진구': '2지역', '성동구': '4지역', '성북구': '2지역', '동대문구': '2지역', '구리시': '2지역', '중원구': '3지역', '성남시 수정구': '3지역', '성남시 분당구': '3지역', '용산구': '4지역', '마포구': '4지역', '중구': '4지역', '종로구': '4지역', '서대문구': '4지역', '은평구': '4지역', '금천구': '5지역', '관악구': '5지역', '구로구': '5지역', '영등포구': '5지역', '양천구': '5지역', '강서구': '5지역', '동작구': '5지역', '동구': '6지역', '연수구': '6지역', '남동구': '6지역', '남구': '6지역', '미추홀구': '6지역', '서구': '6지역', '계양구': '6지역', '부평구': '6지역', '용인시 기흥구': '7지역', '용인시 수지구': '7지역', '수원시 영통구': '7지역'}


### 테크니션 불러오기_(현재는 가상설정)

In [6]:
# technician 배치가능지역 불러오기 
df = pd.read_csv("desktop/테크니션_revise1.csv", header= None)
df.columns = ["name", "location"]
df["location"] = df["location"].str.split(",")

df

,name,location
0,테크니션1,"[동구, 연수구, 남동구, 남구, 미추홀구, 서구, 계양구, 부평구]"
1,테크니션2,"[동구, 연수구, 남동구, 남구, 미추홀구, 서구, 계양구, 부평구]"
2,테크니션3,"[금천구, 관악구, 구로구, 영등포구, 양천구, 강서구, 동작구]"
3,테크니션4,"[금천구, 관악구, 구로구, 영등포구, 양천구, 강서구, 동작구]"
4,테크니션5,"[금천구, 관악구, 구로구, 영등포구, 양천구, 강서구, 동작구]"
5,테크니션6,"[금천구, 관악구, 구로구, 영등포구, 양천구, 강서구, 동작구]"
6,테크니션7,"[금천구, 관악구, 구로구, 영등포구, 양천구, 강서구, 동작구]"
7,테크니션8,"[용산구, 마포구, 중구, 종로구, 서대문구, 은평구, 성동구]"
8,테크니션9,"[용산구, 마포구, 중구, 종로구, 서대문구, 은평구, 성동구]"
9,테크니션10,"[용산구, 마포구, 중구, 종로구, 서대문구, 은평구, 성동구]"


In [7]:
# 테스트를 위해 강남구만 선택하기 
# 배치가능 1, 불가능 0 binary 형태 테이블로 정제 
index = []
for k, i in enumerate(df["location"]): 
    if "강남구" in i:
        index.append(k)

df = df.loc[index]
df1 = pd.DataFrame(index=df["name"], columns=set(df["location"].sum()))

for i, k in zip(df["name"], df["location"]):
    df1.loc[i, k] = 1
df1 = df1.fillna(0)

df1

,서초구,송파구,강남구
name,,,
테크니션22,1,1,1
테크니션23,1,1,1
테크니션24,1,1,1
테크니션25,1,1,1
테크니션26,1,1,1
테크니션27,1,1,1


### 원하는지역 고객선택하기 

In [8]:
#강남3구, 용산 임의선택 

order= []
for k, i in enumerate(df_order["지역구분"]): 
    if "1지역" in i: 
           order.append(k)

df_order = df_order.iloc[order]

# 고객의 수 
cust_number = len(df_order)
# 테크니션 수
tech_number = df_order.TB_USER_ID_EMPLOYEE.nunique()
print(f"총 고객의 수 : {cust_number} 명")
print(f"총 테크니션 수 : {tech_number} 명")

총 고객의 수 : 18 명
총 테크니션 수 : 7 명


### 테크니션수 임의조정 
테스트를 위해 테크니션 수를 임의로 지정해 줍니다. 

In [9]:
# 테크니션 수 임의 조정 : 6명 설정 
tech_number = len(df)

### Duration 생성 (이동시간)
네이버 api로 이동시간을 받아옵니다. 

In [10]:
# 위경도 데이터타입변경 
df_order['USER_LAT'] = df_order['USER_LAT'].astype(float)
df_order['USER_LNG'] = df_order['USER_LNG'].astype(float)

# 데이터 시간순 재정렬 
df_order = df_order.sort_values(by = "ORDER_START_DATE")
df_order = pd.concat([df_order[:tech_number], df_order], axis=0)
# 테크니션 위치 위도/경도는 사용하지 않으므로 임의로 0으로 지정 
df_order[:tech_number] = 0   

# 네이버 API 사용
# 위도, 경도축출
import time
from urllib.request import urlopen, Request
import json
distance = []
duration = []

cnt = 0
error = 0
index = []
situ = []

# # Naver api (변경필요)
client_id = "5nq2hgq4ct"
client_pw = "eAcaE6E3aJaqTTNK1dtZaQy6XkmzwtcwQGDxul0H"

distance = []
for k, i in zip(df_order['USER_LAT'], df_order["USER_LNG"]):
    for kk, ii in zip(df_order['USER_LAT'], df_order["USER_LNG"]):
        distance.append([k, i, kk, ii])
        cnt += 1
        print(k, i, kk, ii, "in progress")

        # 위경도 동일할시 이동시간은 0 
        if (k == kk) & (i == ii):
            distance.append(0)
            duration.append(0)
            situ.append([k, i, kk, ii])
            
        # 위경도가 0을 포함할 시 이동시간은 0 
        elif (k == 0) | (kk == 0):
            distance.append(0)
            duration.append(0)
            situ.append([k, i, kk, ii])

        else:
            #print(k, i, kk, ii)
            url = f"https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving?start={i},{k}&goal={ii},{kk}&option=trafast"

            request = Request(url)
            request.add_header("X-NCP-APIGW-API-KEY-ID", client_id)
            request.add_header("X-NCP-APIGW-API-KEY", client_pw)

            response = urlopen(request)

            response_body = response.read().decode('utf-8')
            response_body = json.loads(response_body)

            situ.append([k, i, kk, ii])

            duration.append(
                round(
                    response_body['route']["trafast"][0]["summary"]["duration"]
                    / 1000 / 60))
            print(
                round(
                    response_body['route']["trafast"][0]["summary"]["duration"]
                    / 1000 / 60))
            distance.append(
                response_body['route']["trafast"][0]["summary"]["distance"] /
                1000)

0.0 0.0 0.0 0.0 in progress
0.0 0.0 0.0 0.0 in progress
0.0 0.0 0.0 0.0 in progress
0.0 0.0 0.0 0.0 in progress
0.0 0.0 0.0 0.0 in progress
0.0 0.0 0.0 0.0 in progress
0.0 0.0 37.480366 127.06383899999999 in progress
0.0 0.0 37.482454 127.00531799999999 in progress
0.0 0.0 37.4751 127.12299999999999 in progress
0.0 0.0 37.505697999999995 127.045441 in progress
0.0 0.0 37.479633 127.14407800000001 in progress
0.0 0.0 37.5148 127.111764 in progress
0.0 0.0 37.468922 127.04066999999999 in progress
0.0 0.0 37.520635 127.10761399999998 in progress
0.0 0.0 37.5043 127.113 in progress
0.0 0.0 37.499794 127.035021 in progress
0.0 0.0 37.496171000000004 127.05891000000001 in progress
0.0 0.0 37.501395 127.03477099999999 in progress
0.0 0.0 37.481234 127.121873 in progress
0.0 0.0 37.523103000000006 127.02698500000001 in progress
0.0 0.0 37.483967 127.01826899999999 in progress
0.0 0.0 37.511169 127.099236 in progress
0.0 0.0 37.529368 127.044133 in progress
0.0 0.0 37.486596999999996 127.014798

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


15
37.480366 127.06383899999999 37.505697999999995 127.045441 in progress
20
37.480366 127.06383899999999 37.479633 127.14407800000001 in progress
27
37.480366 127.06383899999999 37.5148 127.111764 in progress
23
37.480366 127.06383899999999 37.468922 127.04066999999999 in progress
20
37.480366 127.06383899999999 37.520635 127.10761399999998 in progress
28
37.480366 127.06383899999999 37.5043 127.113 in progress
17
37.480366 127.06383899999999 37.499794 127.035021 in progress
24
37.480366 127.06383899999999 37.496171000000004 127.05891000000001 in progress
13
37.480366 127.06383899999999 37.501395 127.03477099999999 in progress
24
37.480366 127.06383899999999 37.481234 127.121873 in progress
16
37.480366 127.06383899999999 37.523103000000006 127.02698500000001 in progress
35
37.480366 127.06383899999999 37.483967 127.01826899999999 in progress
21
37.480366 127.06383899999999 37.511169 127.099236 in progress
22
37.480366 127.06383899999999 37.529368 127.044133 in progress
29
37.480366 1

#### 이동시간 삼각행렬 생성 

In [11]:
distance = np.array(duration)
distance = distance.reshape(len(df_order), -1)

colname = []
for i in range(len(df_order)):
    colname.append(f"Cust{i+1}")

dist = pd.DataFrame(columns=colname, data=distance, index=colname)


addtional_duration = 10 

dist[dist != 0] += addtional_duration 

# 삼각행렬 만들기
for i, l1 in enumerate(dist):
    for j, l2 in enumerate(dist):
        if i < j:  
            dist.loc[l1, l2] = 0 
dist = dist.T
dist 

,Cust1,Cust2,Cust3,Cust4,Cust5,Cust6,Cust7,Cust8,Cust9,Cust10,...,Cust15,Cust16,Cust17,Cust18,Cust19,Cust20,Cust21,Cust22,Cust23,Cust24
Cust1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cust2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cust3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cust4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cust5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cust6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Cust7,0,0,0,0,0,0,0,31,26,27,...,28,28,20,31,25,40,30,29,33,32
Cust8,0,0,0,0,0,0,0,0,49,34,...,50,32,36,32,48,39,17,48,46,17
Cust9,0,0,0,0,0,0,0,0,0,36,...,30,41,32,46,13,44,43,31,32,44
Cust10,0,0,0,0,0,0,0,0,0,0,...,38,18,26,20,39,28,31,31,26,32


### Customer table 생성 

In [12]:
df_order.ORDER_START_DATE =  pd.to_datetime(df_order.ORDER_START_DATE)

customer_name = []
for i in range(cust_number):
    customer_name.append("C" + str(i + 1) + ":" + "point" +
                         str(i + tech_number))

point_bag = []
for i in range(len(df_order)): 
    point_bag.append(f'point{i+1}')
    
customer_match = [] 
point_list = []

# 고객이름:위치 매치 만들기 
for i in range(cust_number):
    customer_match.append("C" + str(i + 1) + ":" + "point" + str(i + tech_number + 1))
    point_list.append("point" + str(tech_number + i +1))

# 고객위치 
C_bag = ["C" + str(i + 1) for i in range(cust_number)]
customer_location = point_list

schedule = pd.DataFrame(
    columns=[customer_match],
    index=["service type", "Sales", "Duration", "Start time", "End time", "rsv", "address"])

df_order = df_order[tech_number:]

schedule.loc["service type"] =  df_order.total_duration.values
schedule.loc["Sales"]  = df_order["price"].values
schedule.loc["Duration"] = df_order.total_duration.values

schedule.loc["Start time"] = df_order.ORDER_START_DATE.dt.time
schedule.loc["Start time"] = str(df_order.ORDER_START_DATE.dt.time)
timelist = [str(i) for i in df_order.ORDER_START_DATE.dt.time]
schedule.loc["Start time"] = timelist
schedule.loc["Start time"] = pd.to_datetime(schedule.loc["Start time"]).dt.time

# 근무시작시간
w_start_time = 10 * 60

schedule.loc["Start time"] = schedule.loc["Start time"].apply(
    lambda x: 60 * x.hour + x.minute - w_start_time)
schedule.loc["End time"] = schedule.loc["Start time"]
schedule.loc["Due time"] = schedule.loc["End time"] + schedule.loc["Duration"]

customer_name = C_bag

schedule.loc["location"] = customer_location
schedule.loc["names"] = customer_name
schedule.loc["rsv"] = df_order.ORDER_START_DATE.dt.time.values
schedule.loc["address"] = df_order.address.values

schedule = schedule.reindex([
    'names', 'location', 'service type', 'Sales', 'Start time', 'End time',
    'Due time', 'Duration', "rsv", "address"
])

schedule

,C1:point7,C2:point8,C3:point9,C4:point10,C5:point11,C6:point12,C7:point13,C8:point14,C9:point15,C10:point16,C11:point17,C12:point18,C13:point19,C14:point20,C15:point21,C16:point22,C17:point23,C18:point24
names,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18
location,point7,point8,point9,point10,point11,point12,point13,point14,point15,point16,point17,point18,point19,point20,point21,point22,point23,point24
service type,90,110,90,90,55,140,90,90,85,150,90,90,90,130,115,100,100,50
Sales,57000,79000,28500,52000,40000,71400,45000,48000,21000,75600,48000,57000,57000,94000,62000,58000,64000,32000
Start time,5,30,70,125,135,165,190,195,240,240,250,320,360,360,365,390,427,480
End time,5,30,70,125,135,165,190,195,240,240,250,320,360,360,365,390,427,480
Due time,95,140,160,215,190,305,280,285,325,390,340,410,450,490,480,490,527,530
Duration,90,110,90,90,55,140,90,90,85,150,90,90,90,130,115,100,100,50
rsv,10:05:00,10:30:00,11:10:00,12:05:00,12:15:00,12:45:00,13:10:00,13:15:00,14:00:00,14:00:00,14:10:00,15:20:00,16:00:00,16:00:00,16:05:00,16:30:00,17:07:00,18:00:00
address,강남구,서초구,송파구,강남구,송파구,송파구,서초구,송파구,송파구,강남구,강남구,강남구,송파구,강남구,서초구,송파구,강남구,서초구


In [13]:
# 유연화 
time_flex = 30 

schedule.loc["Start time"] = schedule.loc["Start time"].apply(lambda x: x - time_flex if x !=0 else x)

schedule = schedule.T

import datetime 

tmp = pd.DataFrame(
    schedule[schedule["rsv"].values >= datetime.time(17, 30, 00)]
    ["location"])["location"].apply(lambda x: x[5:]).astype(int).values
schedule = schedule.T

#출퇴근시간 해당 예약타임 
dist[dist.iloc[:, tmp.min()-1:tmp.max()] != 0] = dist[dist.iloc[:, tmp.min()-1:tmp.max()] != 0] * 2

schedule.drop("rsv", axis = 0, inplace = True)

tech_name = []
for i in range(tech_number):
    tech_name.append(f"technician{i+1}")

depot = point_bag[:tech_number]

tech = pd.DataFrame(columns=tech_name,
                    index=["Minutes", "Depot"])

# 총근무시간 
working_time = 600 

tech.loc["Minutes"] = working_time
tech.loc["Depot"] = depot

tech

,technician1,technician2,technician3,technician4,technician5,technician6
Minutes,600,600,600,600,600,600
Depot,point1,point2,point3,point4,point5,point6


#### 테크니션 야근설정 

In [14]:
# 야근
#night_shift = 1 
#tech.loc["Minutes"][:10] = 730

#tech

#### 테크니션 지역설정 

In [15]:
# 테크니션 이름 딕셔너리 관리 
tech_to_names = {i:k for i in df1.index for k in tech.columns}
name_to_techs = {k:i for i in df1.index for k in tech.columns}

df1.index = tech.columns

schedule = schedule.T
# 지역배정설정 
schedule["coveredby"] = np.nan

tech_avail = []
for i in range(len(schedule)): 
    loc = schedule["address"][i]
    tech_avail.append(list(df1[df1[loc] == 1].index))
    #schedule["available_technician"][i] = tech_as

schedule["coveredby"] = tech_avail

schedule

,names,location,service type,Sales,Start time,End time,Due time,Duration,address,coveredby
C1:point7,C1,point7,90,57000,-25,5,95,90,강남구,"[technician1, technician2, technician3, techni..."
C2:point8,C2,point8,110,79000,0,30,140,110,서초구,"[technician1, technician2, technician3, techni..."
C3:point9,C3,point9,90,28500,40,70,160,90,송파구,"[technician1, technician2, technician3, techni..."
C4:point10,C4,point10,90,52000,95,125,215,90,강남구,"[technician1, technician2, technician3, techni..."
C5:point11,C5,point11,55,40000,105,135,190,55,송파구,"[technician1, technician2, technician3, techni..."
C6:point12,C6,point12,140,71400,135,165,305,140,송파구,"[technician1, technician2, technician3, techni..."
C7:point13,C7,point13,90,45000,160,190,280,90,서초구,"[technician1, technician2, technician3, techni..."
C8:point14,C8,point14,90,48000,165,195,285,90,송파구,"[technician1, technician2, technician3, techni..."
C9:point15,C9,point15,85,21000,210,240,325,85,송파구,"[technician1, technician2, technician3, techni..."
C10:point16,C10,point16,150,75600,210,240,390,150,강남구,"[technician1, technician2, technician3, techni..."


In [16]:
schedule = schedule.T

named = []
for k, i in enumerate(schedule.loc["address"]):
    named.append(df1[df1[i] == 1].index)

service_list = {}
for k, i in enumerate(schedule.loc["service type"].unique()):
    service_list.update({k: i})

duration_backup = schedule.loc["service type"].values

schedule

,C1:point7,C2:point8,C3:point9,C4:point10,C5:point11,C6:point12,C7:point13,C8:point14,C9:point15,C10:point16,C11:point17,C12:point18,C13:point19,C14:point20,C15:point21,C16:point22,C17:point23,C18:point24
names,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18
location,point7,point8,point9,point10,point11,point12,point13,point14,point15,point16,point17,point18,point19,point20,point21,point22,point23,point24
service type,90,110,90,90,55,140,90,90,85,150,90,90,90,130,115,100,100,50
Sales,57000,79000,28500,52000,40000,71400,45000,48000,21000,75600,48000,57000,57000,94000,62000,58000,64000,32000
Start time,-25,0,40,95,105,135,160,165,210,210,220,290,330,330,335,360,397,450
End time,5,30,70,125,135,165,190,195,240,240,250,320,360,360,365,390,427,480
Due time,95,140,160,215,190,305,280,285,325,390,340,410,450,490,480,490,527,530
Duration,90,110,90,90,55,140,90,90,85,150,90,90,90,130,115,100,100,50
address,강남구,서초구,송파구,강남구,송파구,송파구,서초구,송파구,송파구,강남구,강남구,강남구,송파구,강남구,서초구,송파구,강남구,서초구
coveredby,"[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni..."


In [17]:
schedule.loc["service type"] = schedule.loc["names"] +  schedule.loc["address"] 

canCover = {i:k for i, k in zip(schedule.loc["names"],schedule.loc["coveredby"])}

product = pd.DataFrame(index=schedule.loc["names"],
                       columns=["Duration"],
                       data=schedule.loc["Duration"].values)

# 일 우선순위 부여 : 우선순위는 동일하게 1로 설정을 한다. 
product["Priority"] = 1  

product.index = schedule.loc["service type"].values 
product

,Duration,Priority
C1강남구,90,1
C2서초구,110,1
C3송파구,90,1
C4강남구,90,1
C5송파구,55,1
C6송파구,140,1
C7서초구,90,1
C8송파구,90,1
C9송파구,85,1
C10강남구,150,1


In [18]:
# 클래스 설정


class Technician():
    def __init__(self, cap, depot, name):
        self.name = name
        self.cap = cap
        self.depot = depot

    def __str__(self):
        return f"Technician: {self.name}\n  Capacity: {self.cap}\n  Depot: {self.depot}"


class Job():
    def __init__(self, name, duration, priority, coveredBy):
        self.name = name
        self.priority = priority
        self.duration = duration
        self.coveredBy = coveredBy  #coveredBy를 새롭게 추가

    def __str__(self):
        about = f"Job: {self.name}\n  Priority: {self.priority}\n  Duration: {self.duration}\n  Covered by: "
        about += ", ".join([t.name for t in self.coveredBy])
        return about


class Customer():
    def __init__(self, name, loc, job, sales, tStart, tEnd, tDue, duration):
        self.name = name
        self.loc = loc
        self.job = job
        self.sales = sales
        self.tStart = tStart
        self.tEnd = tEnd
        self.tDue = tDue
        self.duration = duration

    def __str__(self):
        coveredBy = ", ".join([t.name for t in self.job.coveredBy])
        return f"Customer: {self.name}\n  Location: {self.loc}\n  Job: {self.job.name}\n  Priority: {self.job.priority}\n  Duration: {self.job.duration}\n  Covered by: {coveredBy}\n  Start time: {self.tStart}\n  End time: {self.tEnd}\n  Due time: {self.tDue}"

In [19]:
schedule

,C1:point7,C2:point8,C3:point9,C4:point10,C5:point11,C6:point12,C7:point13,C8:point14,C9:point15,C10:point16,C11:point17,C12:point18,C13:point19,C14:point20,C15:point21,C16:point22,C17:point23,C18:point24
names,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18
location,point7,point8,point9,point10,point11,point12,point13,point14,point15,point16,point17,point18,point19,point20,point21,point22,point23,point24
service type,C1강남구,C2서초구,C3송파구,C4강남구,C5송파구,C6송파구,C7서초구,C8송파구,C9송파구,C10강남구,C11강남구,C12강남구,C13송파구,C14강남구,C15서초구,C16송파구,C17강남구,C18서초구
Sales,57000,79000,28500,52000,40000,71400,45000,48000,21000,75600,48000,57000,57000,94000,62000,58000,64000,32000
Start time,-25,0,40,95,105,135,160,165,210,210,220,290,330,330,335,360,397,450
End time,5,30,70,125,135,165,190,195,240,240,250,320,360,360,365,390,427,480
Due time,95,140,160,215,190,305,280,285,325,390,340,410,450,490,480,490,527,530
Duration,90,110,90,90,55,140,90,90,85,150,90,90,90,130,115,100,100,50
address,강남구,서초구,송파구,강남구,송파구,송파구,서초구,송파구,송파구,강남구,강남구,강남구,송파구,강남구,서초구,송파구,강남구,서초구
coveredby,"[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni...","[technician1, technician2, technician3, techni..."


### Data Storing

In [20]:
tech.loc["names", :] = tech_name

technicians = []
for i in iter(tech.T.values):
    thisTech = Technician(*i)
    technicians.append(thisTech)

product = product.reset_index().rename({'index': 'Name'}, axis=1)

schedule.loc["coveredby"] = schedule.loc["coveredby"].apply(lambda x: ','.join(x))

product["coveredBy"] = schedule.loc["coveredby"].values

jobs = []
for i in iter(product.values):
    thisJob = Job(*i)
    jobs.append(thisJob)

locations = point_bag
dist1 = {(l, l): 0 for l in locations}

for i, l1 in enumerate(locations):
    for j, l2 in enumerate(locations):
        if i < j: 
            dist1[l1, l2] = dist.iloc[i, j] 
            dist1[l2, l1] = dist1[l1, l2]  

schedule = schedule.drop(["address", "coveredby"], axis = 0)

customers = []
for i in iter(schedule.T.values):
    thisCustomer = Customer(*i)
    customers.append(thisCustomer)

### Variables 설정

In [21]:
K = [k.name for k in technicians]  # technitian name
C = [j.name for j in customers]  # Custmer name
J = [j.loc for j in customers]  # Customer location set

L = point_bag 
D = list(set([t.depot for t in technicians]))
cap = {k.name: k.cap for k in technicians}
loc = {j.name: j.loc for j in customers}
depot = {k.name: k.depot for k in technicians}
dur = {j.name: j.duration for j in customers}
tStart = {j.name: j.tStart for j in customers}
tEnd = {j.name: j.tEnd for j in customers}
tDue = {j.name: j.tDue for j in customers}
priority = {i.name: 1 for i in customers}

### Creating Model & Constraints

In [22]:
### Create model
m = gp.Model("trs0")  # 모델 선언

### Decision variables
# Customer-technician assignment

# Customer-Technition 조합 (배정)
x = m.addVars(C, K, vtype=GRB.BINARY, name="x")

# Technician assignment
u = m.addVars(K, vtype=GRB.BINARY, name="u")

y = m.addVars(L, L, K, vtype=GRB.BINARY, name="y")

# Technician cannot leave or return to a depot that is not its base
for k in technicians:
    for d in D:
        if k.depot != d:
            for i in L:
                y[i, d, k.name].ub = 0
                y[d, i, k.name].ub = 0

# Start time of service
# tj≥0 : This variable determines the time to arrive or start the service at location  j∈J .
t = m.addVars(L, ub=working_time, name="t") 

# Lateness of service
# zj≥0 : This variable determines the lateness of completing job j∈J.
z = m.addVars(C, name="z")

# Artificial variables to correct time window upper and lower limits
# xaj,xbj≥0 : Correction to earliest and latest time to start the service for job  j∈J .
xa = m.addVars(C, name="xa")
xb = m.addVars(C, name="xb")

# Unfilled jobs
# gj : This variable is equal 1 if job  j∈J  cannot be filled, and 0 otherwise.
g = m.addVars(C, vtype=GRB.BINARY, name="g")

### Constraints1

# A technician must be assigned to a job, or a gap is declared (1)

# ∑x(j,k)+g(j)=1, ∀j∈J
# k∈K(j)

# A technician must be assigned to a job, or a gap is declared (1)
m.addConstrs((gp.quicksum(x[j, k] for k in canCover[j]) + g[j] == 1 for j in C), name="assignToJob")


### Constraints2

# At most one technician can be assigned to a job (2)
#∑x(j,k)≤1, ∀j∈J
#k∈K

m.addConstrs((x.sum(j, '*') <= 1 for j in C), name="assignOne")

# Technician capacity constraints (3)

#  ∑p(j)⋅x(j,k)+∑(i∈L)∑(j∈L)τ(i,j)⋅y(i,j,k)≤W(k)⋅u(k), ∀k∈K
#(j∈J)

capLHS = {k : gp.quicksum(dur[j] * x[j,k] for j in C) +\
    gp.quicksum(dist1[i,j]*y[i,j,k] for i in L for j in L) for k in K}
m.addConstrs((capLHS[k] <= cap[k] * u[k] for k in K), name="techCapacity")

# Technician tour constraints (4 and 5)

# For each technician and job, we ensure that if the technician is assigned to the job,
# then the technician must travel to another location (to form a tour).

m.addConstrs((y.sum('*', loc[j], k) == x[j,k] for k in K for j in C),\
    name="techTour1")
m.addConstrs((y.sum(loc[j], '*', k) == x[j,k] for k in K for j in C),\
    name="techTour2")

# constraints 6, 7

# Same depot constraints (6 and 7)
# Same depot: For each technician and depot, we ensure that a technician,
# if assigned to any job, must depart from and return to the service center (depot)
# where the technician is based.

m.addConstrs((gp.quicksum(y[j,depot[k],k] for j in J) == u[k] for k in K),\
    name="sameDepot1")
m.addConstrs((gp.quicksum(y[depot[k],j,k] for j in J) == u[k] for k in K),\
    name="sameDepot2")

Academic license - for non-commercial use only - expires 2021-03-21
Using license file /Users/dukwoonam/gurobi.lic


{'technician1': <gurobi.Constr *Awaiting Model Update*>,
 'technician2': <gurobi.Constr *Awaiting Model Update*>,
 'technician3': <gurobi.Constr *Awaiting Model Update*>,
 'technician4': <gurobi.Constr *Awaiting Model Update*>,
 'technician5': <gurobi.Constr *Awaiting Model Update*>,
 'technician6': <gurobi.Constr *Awaiting Model Update*>}

- **Temporal relationship:** For each location and job, we ensure the temporal relationship between two consecutive jobs served by the same technician. That is, if a technician $k$ travels from job $i$ to job $j$, then the start of the service time at job $j$ must be no less than the completion time of job $i$ plus the travel time from job $i$ to job $j$. 

\begin{equation}
t_{j} \geq t_{i} + p_{i} + \tau_{i,j} - M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) \quad \forall i \in L, j \in J
\tag{8}
\end{equation}

**Note**: Observe that if the technician $k$ travels from the location of job $i$ to the location of job $j$, then
$\sum_{k \in K}  y_{i,j,k} = 1$. Therefore, $M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) = 0$, and the constraint 
$t_{j} \geq t_{i} + p_{i} + \tau_{i,j}$ would be properly enforced. Now consider the case where the technician $k$ does not travel  from the location of job $i$ to the location of job $j$. Hence, $\sum_{k \in K}  y_{i,j,k} = 0$ and 
$M \cdot (1 - \sum_{k \in K}  y_{i,j,k}) = M$. In this case, this constraint becomes 
$t_{j} \geq t_{i} + p_{i} + \tau_{i,j} - M$. But $M$ is a very large number, then $t_{i} + p_{i} + \tau_{i,j} - M < 0$ and 
since $t_{j} \geq 0$, this constraint is redundant.

In [23]:
# Temporal constraints (8) for customer locations
# Temporal relationship: For each location and job,
# we ensure the temporal relationship between two consecutive jobs served by the same technician.
# That is, if a technician  k  travels from job  i  to job  j ,
# then the start of the service time at job  j  must be no less than the completion time of job  i
# plus the travel time from job  i  to job  j .

M = {(i, j): w_start_time + dur[i] + dist1[loc[i], loc[j]] for i in C for j in C}
m.addConstrs((t[loc[j]] >= t[loc[i]] + dur[i] + dist1[loc[i], loc[j]]\
    - M[i,j]*(1 - gp.quicksum(y[loc[i],loc[j],k] for k in K))\
    for i in C for j in C), name="tempoCustomer")

# # Temporal constraints (8) for depot locations
M = {(i, j): w_start_time + dist1[i, loc[j]] for i in D for j in C}
m.addConstrs((t[loc[j]] >= t[i] + dist1[i, loc[j]]\
    - M[i,j]*(1 - y.sum(i,loc[j],'*')) for i in D for j in C),\
    name="tempoDepot")

{('point4', 'C1'): <gurobi.Constr *Awaiting Model Update*>,
 ('point4', 'C2'): <gurobi.Constr *Awaiting Model Update*>,
 ('point4', 'C3'): <gurobi.Constr *Awaiting Model Update*>,
 ('point4', 'C4'): <gurobi.Constr *Awaiting Model Update*>,
 ('point4', 'C5'): <gurobi.Constr *Awaiting Model Update*>,
 ('point4', 'C6'): <gurobi.Constr *Awaiting Model Update*>,
 ('point4', 'C7'): <gurobi.Constr *Awaiting Model Update*>,
 ('point4', 'C8'): <gurobi.Constr *Awaiting Model Update*>,
 ('point4', 'C9'): <gurobi.Constr *Awaiting Model Update*>,
 ('point4', 'C10'): <gurobi.Constr *Awaiting Model Update*>,
 ('point4', 'C11'): <gurobi.Constr *Awaiting Model Update*>,
 ('point4', 'C12'): <gurobi.Constr *Awaiting Model Update*>,
 ('point4', 'C13'): <gurobi.Constr *Awaiting Model Update*>,
 ('point4', 'C14'): <gurobi.Constr *Awaiting Model Update*>,
 ('point4', 'C15'): <gurobi.Constr *Awaiting Model Update*>,
 ('point4', 'C16'): <gurobi.Constr *Awaiting Model Update*>,
 ('point4', 'C17'): <gurobi.Const

In [24]:
# Time window: For each job  j∈J  ensure that the time window for the job is satisfied.
# t : 실제 서비스 시작타임

# Time window constraints (9 and 10)
m.addConstrs((t[loc[j]] + xa[j] >= tStart[j] for j in C), name="timeWinA")
m.addConstrs((t[loc[j]] - xb[j] <= tEnd[j] for j in C), name="timeWinB")

{'C1': <gurobi.Constr *Awaiting Model Update*>,
 'C2': <gurobi.Constr *Awaiting Model Update*>,
 'C3': <gurobi.Constr *Awaiting Model Update*>,
 'C4': <gurobi.Constr *Awaiting Model Update*>,
 'C5': <gurobi.Constr *Awaiting Model Update*>,
 'C6': <gurobi.Constr *Awaiting Model Update*>,
 'C7': <gurobi.Constr *Awaiting Model Update*>,
 'C8': <gurobi.Constr *Awaiting Model Update*>,
 'C9': <gurobi.Constr *Awaiting Model Update*>,
 'C10': <gurobi.Constr *Awaiting Model Update*>,
 'C11': <gurobi.Constr *Awaiting Model Update*>,
 'C12': <gurobi.Constr *Awaiting Model Update*>,
 'C13': <gurobi.Constr *Awaiting Model Update*>,
 'C14': <gurobi.Constr *Awaiting Model Update*>,
 'C15': <gurobi.Constr *Awaiting Model Update*>,
 'C16': <gurobi.Constr *Awaiting Model Update*>,
 'C17': <gurobi.Constr *Awaiting Model Update*>,
 'C18': <gurobi.Constr *Awaiting Model Update*>}

- **Lateness constraint:** For each job $j \in J$ calculate the lateness of the job.

\begin{equation}
z_{j} \geq t_{j} + p_{j} - dd_{j} \quad \forall j \in J
\tag{11}
\end{equation}

- Note that since the lateness decision variable $z_{j}$ is non-negative, there is no benefit to complete a job before its due date; on the other hand, since the objective function minimizes the total weighted lateness, Constraint (11) should always be binding

In [25]:
# Note: To discourage that the time window of a job is violated,
# we associate the penalty of ( 0.01⋅πj⋅M ) to the correction variables  xaj,xbj .

# Lateness constraint (11)
# Lateness constraint: For each job  j∈J  calculate the lateness of the job.
# Note that since the lateness decision variable  zj  is non-negative, there is no benefit to complete
# a job before its due date; on the other hand, since the objective function minimizes
# the total weighted lateness, Constraint (11) should always be binding.

# Lateness of service
# zj≥0 : This variable determines the lateness of completing job j∈J.
# z = m.addVars(C, name="z")

m.addConstrs((z[j] >= t[loc[j]] + dur[j] - tDue[j] for j in C),\
    name="lateness")

{'C1': <gurobi.Constr *Awaiting Model Update*>,
 'C2': <gurobi.Constr *Awaiting Model Update*>,
 'C3': <gurobi.Constr *Awaiting Model Update*>,
 'C4': <gurobi.Constr *Awaiting Model Update*>,
 'C5': <gurobi.Constr *Awaiting Model Update*>,
 'C6': <gurobi.Constr *Awaiting Model Update*>,
 'C7': <gurobi.Constr *Awaiting Model Update*>,
 'C8': <gurobi.Constr *Awaiting Model Update*>,
 'C9': <gurobi.Constr *Awaiting Model Update*>,
 'C10': <gurobi.Constr *Awaiting Model Update*>,
 'C11': <gurobi.Constr *Awaiting Model Update*>,
 'C12': <gurobi.Constr *Awaiting Model Update*>,
 'C13': <gurobi.Constr *Awaiting Model Update*>,
 'C14': <gurobi.Constr *Awaiting Model Update*>,
 'C15': <gurobi.Constr *Awaiting Model Update*>,
 'C16': <gurobi.Constr *Awaiting Model Update*>,
 'C17': <gurobi.Constr *Awaiting Model Update*>,
 'C18': <gurobi.Constr *Awaiting Model Update*>}

### Objective Function

- **Minimize lateness:** The Objective function is to minimize the total weighted lateness of all the jobs. 

\begin{equation}
\sum_{j \in J} \pi_{j} \cdot z_{j} + \sum_{j \in J} 0.01 \cdot \pi_{j} \cdot M(xa_{j} + xb_{j}) + 
\sum_{j \in J} \pi_{j} \cdot M \cdot g_{j}
\tag{0}
\end{equation}

**Note**: We want to treat the constraints of filling the jobs demand and the constraints about starting the jobs within a time window as soft constraints. Soft constraints can be violated, but the violation will incur in a huge penalty. To consider that each job has different priority, we use the priority penalty for the calculation of the penalty associated with a violation of soft constraints. We assume that not filling jobs demand greatly deteriorate customer satisfaction, consequently should incur the largest penalties associated with the soft constraints. Recall that the value of the paramter $M$ is a large number, then the penalty associated with not filling a job is determined as follows: $\pi_{j} \cdot M$. The time windows constraints violation also deteriorates customer satisfaction but to a lesser degree than the penalty associated with violating the time windows constraints. The time window constraints are determined as follows: $0.01 \cdot \pi_{j} \cdot M$.

In [26]:
### Objective function
# Minimize lateness: The Objective function is to minimize the total weighted lateness of all the jobs.

M = 8100 # 패널티 상수 (Big M)

# penalty wegiht
pw = 0.01

m.setObjective(
    z.prod(priority) + gp.quicksum(pw * M * priority[j] * (xa[j] + xb[j])
                                   for j in C) +
    gp.quicksum(M * priority[j] * g[j] for j in C), GRB.MINIMIZE)

m.write("TRS0.lp")
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 756 rows, 3666 columns and 11340 nonzeros
Model fingerprint: 0x6acd1fcc
Variable types: 78 continuous, 3588 integer (3588 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+02]
  Objective range  [1e+00, 8e+03]
  Bounds range     [1e+00, 6e+02]
  RHS range        [1e+00, 6e+02]
Found heuristic solution: objective 456192.00000
Presolve removed 180 rows and 1432 columns
Presolve time: 0.11s
Presolved: 576 rows, 2234 columns, 8906 nonzeros
Variable types: 68 continuous, 2166 integer (2166 binary)

Root relaxation: objective -2.910383e-11, 408 iterations, 0.02 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0   39 456192.000    0.00000   100%     -    0s
H    0     0                    89100.

In [27]:
### 결과 리포트 
# 1. 배정현황 
# 2. 경로 
# 3. 테크니션 활용도 (Utilization: 활동시간 / 근무시간)

report = []
for j in customers:
    if g[j.name].X > 0.5:
        jobStr = "Nobody assigned to {} ({}) in {}".format(
            j.name, j.job, j.loc)
    else:
        for k in K:
            if x[j.name, k].X > 0.5:
                jobStr = "{} assigned to {} ({}) in {}. Start at t={:.2f}.".format(
                    k, j.name, j.name, j.loc, t[j.loc].X)
                if z[j.name].X > 1e-6:
                    jobStr += " {:.2f} minutes late.".format(z[j.name].X)
                if xa[j.name].X > 1e-6:
                    jobStr += " Start time corrected by {:.2f} minutes.".format(
                        xa[j.name].X)
                if xb[j.name].X > 1e-6:
                    jobStr += " End time corrected by {:.2f} minutes.".format(
                        xb[j.name].X)
    print(jobStr)
    report.append(jobStr)

# Technicians

target = []
route_use = []
not_used = []
source = []
tech_f = []
dist_base = []

print("")
for k in technicians:
    if u[k.name].X > 0.5:
        cur = k.depot
        route = k.depot
        while True:
            for j in customers:
                if y[cur, j.loc, k.name].X > 0.5:
                    route += " -> {} (dist1={}, t={:.2f}, proc={})".format(
                        j.loc, dist1[cur, j.loc], t[j.loc].X, j.duration)

                    dist_base.append(dist1[cur, j.loc])
                    cur = j.loc
                    source.append(cur)
                    tech_f.append(k.name)
            for i in D:
                if y[cur, i, k.name].X > 0.5:
                    route += " -> {} (dist1={})".format(i, dist1[cur, i])
                    cur = i
                    target.append(cur)
                    #dist_base.append(dist1[cur, i])
                    break
            if cur == k.depot:
                break
        print("{}'s route: {}".format(k.name, route))
        route_use.append([k.name, route])
    else:
        print("{} is not used".format(k.name))
        not_used.append(k.name)

util_ = []
for k in K:
    used = float(capLHS[k].getValue())
    total = float(cap[k])
    util = used / float(cap[k]) if float(cap[k]) > 0 else 0
    print("{}'s utilization is {:.2%} ({:.2f}/{:.2f})".format(k, util,\
        used, float(cap[k])))
    util_.append([k, util, used, float(cap[k])])
totUsed = sum(capLHS[k].getValue() for k in K)
totCap = sum(float(cap[k]) for k in K)
totUtil = totUsed / totCap if totCap > 0 else 0
print("Total technician utilization is {:.2%} ({:.2f}/{:.2f})".format(
    totUtil, totUsed, totCap))

technician2 assigned to C1 (C1) in point7. Start at t=0.00.
technician5 assigned to C2 (C2) in point8. Start at t=0.00.
technician4 assigned to C3 (C3) in point9. Start at t=40.00.
technician3 assigned to C4 (C4) in point10. Start at t=95.00.
technician6 assigned to C5 (C5) in point11. Start at t=105.00.
technician1 assigned to C6 (C6) in point12. Start at t=135.00.
technician5 assigned to C7 (C7) in point13. Start at t=160.00.
technician2 assigned to C8 (C8) in point14. Start at t=165.00.
technician6 assigned to C9 (C9) in point15. Start at t=235.00.
technician3 assigned to C10 (C10) in point16. Start at t=228.61.
technician4 assigned to C11 (C11) in point17. Start at t=220.00.
technician5 assigned to C12 (C12) in point18. Start at t=305.00.
technician4 assigned to C13 (C13) in point19. Start at t=341.00.
technician2 assigned to C14 (C14) in point20. Start at t=360.00.
technician6 assigned to C15 (C15) in point21. Start at t=365.00.
technician1 assigned to C16 (C16) in point22. Start 

In [28]:
# 결과정리 
tech_start = pd.DataFrame(report)[0].str.split(expand = True)[[0, 9]]
tech_base = pd.DataFrame(report)[0].str.split(expand = True)[[0, 6]]
k = pd.DataFrame(columns= ["Src", "Tech"])
k.Src = source 
k.Tech = tech_f
#tech_f

k["Target"] = source

k.Target = k.Target.shift(-1)
k.groupby("Tech")[["Src", "Target"]]

k2 = k.drop(k.Tech.drop_duplicates(keep = 'last').index, axis = 0)

k2 = k2[["Src", "Target", "Tech"]]
k2.columns = ["Source", "Target", "Tech"]

k2.Source = k2.Source.apply(lambda x : x[5:])
k2.Target = k2.Target.apply(lambda x : x[5:])

df_order["location"] = schedule.loc["location"].values
k3 = df_order[["location", "USER_LAT", "USER_LNG", "TB_USER_ID"]]
k3["Id"] = k3.location.apply(lambda x: x[5:])

loc_dict = {pp:(ii, kk) for pp, ii, kk in zip(k3["Id"], k3["USER_LAT"], k3["USER_LNG"])}

k2["source_loc"] = k2.Source.map(loc_dict)
k2["target_loc"] = k2.Target.map(loc_dict)

k2["source_lat"] = k2["source_loc"].apply(lambda x : x[0])
k2["source_lng"] = k2["source_loc"].apply(lambda x : x[1])
k2["target_lat"] = k2["target_loc"].apply(lambda x : x[0])
k2["target_lng"] = k2["target_loc"].apply(lambda x : x[1])

k2 = k2.drop(["source_loc", "target_loc"], axis = 1)

schedule = schedule.T
schedule["appointed"] = df_order["ORDER_START_DATE"].dt.time.values
tmp = {i:k for i, k in zip(schedule["location"], schedule["appointed"])}
schedule = schedule.T
k3["Rsrv_time"] = k3['location'].map(tmp)

#k2.to_csv("desktop/gangnam.csv")

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

tmp2 = {i:k for i, k in zip(k2["Source"], k2["Tech"])}
tmp3 = {i:k for i, k in zip(k2["Target"], k2["Tech"])}
tmp2.update(tmp3)
k3["Arranged"] = k3["Id"].map(tmp2)
k3 = k3.sort_values(by = ["Arranged", "Rsrv_time"])
k3.head()

dist_base = pd.DataFrame(dist_base)
k3["duration"] = dist_base[dist_base[0]!=0]

tmp2 = dist_base[dist_base[0]!=0]
k3["duration"] = dist_base

tmp = pd.DataFrame(dist_base)
k2["duration"] = tmp[tmp[0] != 0][0].values

schedule = schedule.T

tmp2 = schedule.set_index("location")["Duration"].to_dict()

k3["proc"] = k3["location"].map(tmp2)

k3["duration"] = dist_base.values

tech_start= tech_start[tech_start[0] != "Nobody"]

tech_start[9] = tech_start[9].apply(lambda x : x[2:])
tech_start[9] = tech_start[9].apply(lambda x: x.split(".")[0])
tech_start[9] = tech_start[9].astype(int)
tech_start = tech_start.sort_values(by = [0, 9])
k3["start_at"] = tech_start[9].values

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

### Result Table 

In [29]:
# 테크니션 경로 & Duration 
k2

,Source,Target,Tech,source_lat,source_lng,target_lat,target_lng,duration
0,12,22,technician1,37.514800,127.111764,37.511169,127.099236,20
2,7,14,technician2,37.480366,127.063839,37.520635,127.107614,34
3,14,20,technician2,37.520635,127.107614,37.523103,127.026985,40
5,10,16,technician3,37.505698,127.045441,37.499794,127.035021,18
6,16,24,technician3,37.499794,127.035021,37.486597,127.014798,52
8,9,17,technician4,37.475100,127.123000,37.496171,127.058910,32
9,17,19,technician4,37.496171,127.058910,37.481234,127.121873,31
11,8,13,technician5,37.482454,127.005318,37.468922,127.040670,33
12,13,18,technician5,37.468922,127.040670,37.501395,127.034771,32
13,18,23,technician5,37.501395,127.034771,37.529368,127.044133,32


In [30]:
# 포인트별 예약시간 & process time  
k3

,location,USER_LAT,USER_LNG,TB_USER_ID,Id,Rsrv_time,Arranged,duration,proc,start_at
34,point12,37.514800,127.111764,35,12,12:45:00,technician1,0,140,135
70,point22,37.511169,127.099236,71,22,16:30:00,technician1,20,100,390
16,point7,37.480366,127.063839,17,7,10:05:00,technician2,0,90,0
40,point14,37.520635,127.107614,41,14,13:15:00,technician2,34,90,165
64,point20,37.523103,127.026985,65,20,16:00:00,technician2,40,130,360
23,point10,37.505698,127.045441,24,10,12:05:00,technician3,0,90,95
44,point16,37.499794,127.035021,45,16,14:00:00,technician3,18,150,228
79,point24,37.486597,127.014798,80,24,18:00:00,technician3,52,50,480
20,point9,37.475100,127.123000,21,9,11:10:00,technician4,0,90,40
48,point17,37.496171,127.058910,49,17,14:10:00,technician4,32,90,220
